<a href="https://colab.research.google.com/github/baibai25/survey/blob/main/faiss_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -V

Python 3.7.11


In [2]:
!wget https://anaconda.org/pytorch/faiss-gpu/1.7.1/download/linux-64/faiss-gpu-1.7.1-py3.7_hf4b0e32_5_cuda10.1_nightly.tar.bz2
!tar xvjf faiss-gpu-1.7.1-py3.7_hf4b0e32_5_cuda10.1_nightly.tar.bz2

--2021-08-19 13:24:07--  https://anaconda.org/pytorch/faiss-gpu/1.7.1/download/linux-64/faiss-gpu-1.7.1-py3.7_hf4b0e32_5_cuda10.1_nightly.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.93.24, 104.17.92.24, 2606:4700::6811:5d18, ...
Connecting to anaconda.org (anaconda.org)|104.17.93.24|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5a15c9cc393ec612061f033a/60c02fb0dcb792e0caecd9e8?response-content-disposition=attachment%3B%20filename%3D%22faiss-gpu-1.7.1-py3.7_hf4b0e32_5_cuda10.1_nightly.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27faiss-gpu-1.7.1-py3.7_hf4b0e32_5_cuda10.1_nightly.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=60&X-Amz-Date=20210819T132407Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEHsaCXVzLWVhc3QtMSJGMEQCIDZ1jYMTulN2jvSqCgbHDIbnJa7RAQlAYRnPZErjDkjtAiBiRM8wpFTZ9I34MVln0IsqFxVqNQciTT0zNe7cHQDbJyqDBAij%2F%2F%2F%2F%2

In [3]:
!cp -r lib/python3.7/site-packages/* /usr/local/lib/python3.7/dist-packages/
!pip install faiss-gpu

### faiss
- https://github.com/facebookresearch/faiss
- facebookresearchが提供する近傍探索ライブラリ
  - 

In [4]:
import faiss
import requests
from io import StringIO
import pandas as pd
import numpy as np

### Load Data and Preprocessing

In [5]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')

# create dataframe
data = pd.read_csv(StringIO(res.text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [6]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [7]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)  # merge them
len(set(sentences))  # together we have ~4.5K unique sentences

4802

In [8]:
#  This isn't a particularly large number, so let's pull in a few more similar datasets.
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [9]:
# each of these dataset have the same structure, so we loop through each creating our sentences data
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    data = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
    # add to columns 1 and 2 to sentences list
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

b'Skipping line 191: expected 3 fields, saw 4\nSkipping line 206: expected 3 fields, saw 4\nSkipping line 295: expected 3 fields, saw 4\nSkipping line 695: expected 3 fields, saw 4\nSkipping line 699: expected 3 fields, saw 4\n'
b'Skipping line 104: expected 3 fields, saw 4\nSkipping line 181: expected 3 fields, saw 4\nSkipping line 317: expected 3 fields, saw 4\nSkipping line 412: expected 3 fields, saw 5\nSkipping line 508: expected 3 fields, saw 4\n'


In [10]:
len(set(sentences))

14505

In [11]:
# remove duplicates and NaN
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [12]:
!pip install sentence-transformers

### Create embedding

In [13]:
from sentence_transformers import SentenceTransformer

# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# create sentence embeddings
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(14504, 768)

In [14]:
d = sentence_embeddings.shape[1]
d

768

### IndexFlatL2
- クエリベクトル`xq`とインデックス付きベクトルの`2 distance`を計算
<img src="https://www.pinecone.io/images/faiss2.png" width="500">


In [15]:
index = faiss.IndexFlatL2(d)
index.is_trained

True

In [16]:
index.add(sentence_embeddings)
index.ntotal

14504

### Learning

In [17]:
# Then search given a query xq and number of nearest neigbors to return k.
k = 4
xq = model.encode(["Someone sprints with a football"])

In [18]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[4750 2785 8004 9922]]
CPU times: user 20.2 ms, sys: 0 ns, total: 20.2 ms
Wall time: 20.6 ms


Returns the top k vectors closest to our query vector xq

In [19]:
for i in I.flatten():
  print(sentences[i])

A group of football players is running in the field
A group of people playing football is running in the field
Two groups of people are playing football
A person playing football is running past an official carrying a football


Extract the numerical vectors

In [20]:
# we have 4 vectors to return (k) - so we initialize a zero array to hold them
vecs = np.zeros((k, d))
# then iterate through each ID from I and add the reconstructed vector to our zero-array
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

print(vecs.shape)
vecs[0][:100]

(4, 768)


array([ 0.01627041,  0.22325905, -0.15037419, -0.30747274, -0.27122447,
       -0.10593199, -0.06460919,  0.04738184, -0.73349029, -0.37657705,
       -0.76762801,  0.16902876,  0.53107709,  0.51176673,  1.14415872,
       -0.08562925, -0.67240089, -0.96637052,  0.02545489, -0.21559836,
       -1.25656605, -0.82982129, -0.09825051, -0.21850808,  0.50610203,
        0.10527963,  0.50396872,  0.65242916, -1.39458692,  0.65847492,
       -0.21525379, -0.22487505,  0.81818372,  0.08464319, -0.76141626,
       -0.28928265, -0.09825819, -0.73046225,  0.07855794, -0.84354645,
       -0.59242064,  0.77471328, -1.20920527, -0.22757955, -1.30733669,
       -0.23081465, -1.31322503,  0.01629083, -0.97285444,  0.19308151,
        0.47424582,  1.18920898, -1.96741343, -0.70061094, -0.29638669,
        0.6053369 ,  0.62407452, -0.70340365, -0.86754245,  0.17673172,
       -0.19170479, -0.02952007,  0.22623567, -0.16695429, -0.80402517,
       -0.45918882,  0.69675535, -0.24928199, -1.01478672, -0.92

### Speed
- IndexFlatL2はクエリベクトル`xq`と他のベクトルを全て計算するため計算コストが高い
<img src="https://www.pinecone.io/images/faiss3.png" width="500">

### Partitioning The Index
- 全ベクトルを予め決められたクラスタに分割し、各クラスタのcentroidを保持
- 全ベクトルとの比較ではなく、重心点との比較を行うことで計算を効率化(近似検索)
  - faissでは元ベクトルデータに対しk-meansを適用

<img src="https://www.pinecone.io/images/faiss4.png" width="500">




In [21]:
"""
- nlist (k-means適用後のクラスタ数)
- 推奨値: nlist = 4 * sqrt(総ベクトル数)
- https://github.com/facebookresearch/faiss/issues/112
"""
nlist = 50  # how many cells
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [22]:
index.is_trained

False

In [23]:
index.train(sentence_embeddings)
index.is_trained  # check if index is now trained

True

In [24]:
index.add(sentence_embeddings)
index.ntotal  # number of embeddings indexed

14504

In [25]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[4750 2785 8004 9922]]
CPU times: user 3.07 ms, sys: 960 µs, total: 4.03 ms
Wall time: 5.3 ms


速度比較
- IndexFlatL2のみ
```
CPU times: user 24.6 ms, sys: 867 µs, total: 25.4 ms
Wall time: 25.3 ms
```

- Partitioning The Index
```
CPU times: user 2.01 ms, sys: 117 µs, total: 2.13 ms
Wall time: 2.62 ms
```

### 検索範囲の指定
- `nprobe`: 一番近い重心点を持つクラスタを見つけた後、その周りに存在するクラスタに対し、何個のクラスタを探索しに行くか

<img src="https://www.pinecone.io/images/faiss5.png" width="500">

- `nprobe==1`の場合、最も距離の近い単一のセルのみを探索
- `nprobe==8`の場合、最も近い8つのセルを探索する
- 推奨値: `nprobe <= nlist and nprobe <= 2048`
  - https://github.com/facebookresearch/faiss/wiki/Faiss-on-the-GPU


In [26]:
# We can increase the number of nearby cells to search too with nprobe.
index.nprobe = 10

In [27]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[4750 2785 8004 9922]]
CPU times: user 6.85 ms, sys: 0 ns, total: 6.85 ms
Wall time: 7.53 ms


### Vector Reconstruction
重要じゃないので省略

In [28]:
index.make_direct_map()
index.reconstruct(7460)[:100]

array([ 0.5503561 ,  0.40827504,  0.93015385,  0.19089277, -0.0445136 ,
        1.5070368 ,  1.002186  , -0.08647869, -0.05652123, -0.2527619 ,
       -0.7749146 , -0.22303693,  0.24650946,  1.2829523 ,  0.25591475,
        0.45262042,  0.23186164, -0.57884836, -0.15543476, -0.34583917,
       -0.14736655,  0.27303725,  0.00183087, -0.91337615, -0.22893305,
       -0.34251732, -0.43643808, -1.1937764 , -1.5120819 , -0.4584275 ,
        0.29406297,  0.5659317 ,  1.0271327 ,  0.2779832 ,  0.37686956,
        0.02090846,  0.16037667, -1.0778058 ,  0.16985407, -0.17192662,
        1.0653883 ,  0.1739881 ,  0.18872412,  0.9679464 ,  0.29139987,
        0.24510363,  1.6072366 , -0.21952882, -0.40819865, -0.7689581 ,
       -0.39662066,  0.04877984,  1.5515394 ,  1.1104519 , -0.21962138,
       -0.6538264 ,  0.79120934, -1.1641527 ,  0.9520466 , -0.02531795,
        0.63145083,  0.45889315,  0.06547829, -0.08572054, -1.1775532 ,
        0.10354088, -0.34775686,  0.5833431 , -0.18781927, -1.20

### Quantization
- 大規模データになるほど全てのベクトルを保持するのは困難
- Product Quantization（PQ）を使用してベクトルを圧縮

<img src="https://www.pinecone.io/images/faiss7.png" width="500">

1. サブベクトルに分割
2. サブベクトルごとにクラスタリングを行う
3. Centroid IDとして保持

In [29]:
m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits)

In [30]:
index.is_trained

False

In [31]:
index.train(sentence_embeddings)

In [32]:
index.add(sentence_embeddings)

In [33]:
index.nprobe = 10  # align to previous IndexIVFFlat nprobe value

In [34]:
%%time
D, I = index.search(xq, k)
print(I)

[[2785 8182 4750 8004]]
CPU times: user 8 µs, sys: 967 µs, total: 975 µs
Wall time: 6.99 ms


In [35]:
[f'{i}: {sentences[i]}' for i in I[0]]

['2785: A group of people playing football is running in the field',
 '8182: A group of football players running down the field.',
 '4750: A group of football players is running in the field',
 '8004: Two groups of people are playing football']

### Ref.
1. [Faiss](https://github.com/facebookresearch/faiss)
2. [Faiss Tutorial: Getting Started With Faiss | Pinecone](https://www.pinecone.io/learn/faiss-tutorial/)
3. [レコメンド#3 GPUで近似近傍探索を行うことで大規模データの計算時間を、12時間から50分へ約1/12に削減したお話](https://www.ariseanalytics.com/activities/report/20210304/)
(図解されていて分かりやすい)